In [10]:
from azureml.core import Workspace
from azureml.core.authentication import MsiAuthentication
msi_auth = MsiAuthentication()

ws = Workspace(subscription_id="7704663e-8944-4300-b951-6462d41ab638", # put in .json
resource_group="arg-rt-pacedev-SpaceTech", # also in .json
workspace_name="aml-WS-spacetech001", # also
auth=msi_auth
)

keyvault = ws.get_default_keyvault()
conn_str = keyvault.get_secret("asartpacedevstvdsm001-connstr")

# Baseline Dataset

In [18]:
import datetime
from dateutil.relativedelta import relativedelta
from vdsm_util import get_blob_client
import pickle as pkl
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import StringIO
import traceback

veg_type = 'pv'

training_path = 'training/traininginput_{}_v2'.format(veg_type)

blob_client = BlobServiceClient.from_connection_string(conn_str)
container_client = blob_client.get_container_client("data")

baseline_df = pd.DataFrame()

blob_names = container_client.list_blobs(name_starts_with=f'training/traininginput_{veg_type}_v2')

for blob_name in blob_names:
    blob_client = get_blob_client(filename=blob_name, conn_str=conn_str)
    df = pd.read_csv(StringIO(blob_client.download_blob().content_as_text()))
    baseline_df = baseline_df.append(df)

baseline_df.drop(columns=['actual_3m', 'actual_6m', 'actual_9m', 'actual_12m', 'lat_4326', 'lon_4326'], inplace=True)

output = baseline_df.to_csv(index=False, encoding='utf-8', header=True)

blob_client = get_blob_client(filename=f'data_drift/baseline/{veg_type}.csv', conn_str=conn_str)
blob_client.upload_blob(output, overwrite=True, blob_type='BlockBlob')

{'etag': '"0x8D9EB5923BBE8BD"',
 'last_modified': datetime.datetime(2022, 2, 8, 23, 17, 13, tzinfo=datetime.timezone.utc),
 'content_md5': None,
 'content_crc64': bytearray(b'y\xb6\x16\x1cty\xae\xc6'),
 'client_request_id': '3f8423d0-8935-11ec-8b2c-1bbaf30259bc',
 'request_id': 'd86ff50d-401e-006f-6c42-1da8b3000000',
 'version': '2020-06-12',
 'version_id': '2022-02-08T23:17:13.3789914Z',
 'date': datetime.datetime(2022, 2, 8, 23, 17, 12, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}

# Pipeline for monthly target dataset

In [19]:
import datetime
from dateutil.relativedelta import relativedelta
from vdsm_util import get_blob_client
import pickle as pkl
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import StringIO
import traceback

end_month = '2021-12'
veg_type = 'pv'

blob_client = BlobServiceClient.from_connection_string(conn_str)
container_client = blob_client.get_container_client("data")

end_month_dt = datetime.date(int(end_month[:4]), int(end_month[5:]), 1)
start_month_dt = datetime.date(int(end_month[:4]), int(end_month[5:]), 1) + relativedelta(months=-12)

months = []
month_dt = start_month_dt
while month_dt < end_month_dt:
    month_dt = month_dt + relativedelta(months=1)
    months.append(str(month_dt)[:7])

target_df = pd.DataFrame()

for month in months:

    blob_names = container_client.list_blobs(name_starts_with=f'{month[:4]}/{month[5:]}/fused/{veg_type}_v1')

    for blob_name in blob_names:
        blob_client = get_blob_client(filename=blob_name, conn_str=conn_str)
        df = pd.read_csv(StringIO(blob_client.download_blob().content_as_text()))
        if datetime.date(int(end_month[:4]), int(end_month[5:]), 1) >= datetime.date(2021, 12, 1):
            try:
                df = df.sample(n=100)
            except:
                pass
        target_df = target_df.append(df)

target_df.drop(columns=['filename', 'lat', 'lon'], inplace=True)
target_df.insert(1, 'month', target_df.iloc[:, 1:13].values.argmax(axis=1) + 1)
target_df.drop(columns=['month1','month2','month3','month4','month5','month6','month7','month8','month9','month10','month11','month12'], inplace=True)

output = target_df.to_csv(index=False, encoding='utf-8', header=True)

blob_client = get_blob_client(filename=f'data_drift/target/{end_month}/{veg_type}.csv', conn_str=conn_str)
blob_client.upload_blob(output, overwrite=True, blob_type='BlockBlob')

In [16]:
target_df

year  month       evi  evi_prev       u10  u10_prev       v10  \
1027  2021.0      1  0.161731  0.145613 -0.374168 -0.741189  0.568633   
810   2021.0      1  0.216871  0.206202 -0.374265 -0.741261  0.569096   
1485  2021.0      1  0.151158  0.137834 -0.375518 -0.742449  0.567098   
124   2021.0      1  0.110989  0.113954 -0.372028 -0.739180  0.571317   
666   2021.0      1  0.113616  0.111097 -0.373928 -0.740946  0.569480   
...      ...    ...       ...       ...       ...       ...       ...   
1553  2021.0     12  0.127316  0.116829  0.148742 -0.317432  0.890340   
3453  2021.0     12  0.119469  0.116295  0.161747 -0.313963  0.899750   
3574  2021.0     12  0.102843  0.099476  0.162737 -0.313865  0.900984   
1600  2021.0     12  0.093391  0.077393  0.154521 -0.315347  0.892800   
1416  2021.0     12  0.160558  0.158906  0.148717 -0.317309  0.889907   

      v10_prev         t2m    t2m_prev  ...  tp_prev2      tp_prev3     swvl1  \
1027 -0.427623  304.672019  302.482641  ...  0.000265  8.689519e-05  0.140543   
810  -0.426850  304.674334  302.483897  ...  0.000266  8.714863e-05  0.141794   
1485 -0.429268  304.668191  302.480237  ...  0.000264  8.611414e-05  0.136085   
124  -0.424634  304.678906  302.486745  ...  0.000268  8.826430e-05  0.148171   
666  -0.426439  304.675267  302.484471  ...  0.000266  8.734362e-05  0.142899   
...        ...         ...         ...  ...       ...           ...       ...   
1553 -0.030981  307.903135  301.037010  ...  0.000001  9.555370e-07  0.073711   
3453 -0.032071  307.842288  300.979045  ...  0.000001  9.555370e-07  0.069593   
3574 -0.032108  307.837776  300.974637  ...  0.000001  9.555370e-07  0.069423   
1600 -0.031621  307.875723  301.011210  ...  0.000001  9.555370e-07  0.071328   
1416 -0.031016  307.903166  301.037099  ...  0.000001  9.555370e-07  0.073559   

      swvl1_prev     co2  co2_prev  season_ch_3mon  season_ch_6mon  \
1027    0.197914  414.77    414.14        0.060024       -0.057032   
810     0.199013  414.77    414.14        0.111496       -0.025241   
1485    0.194098  414.77    414.14        0.088653       -0.035169   
124     0.204465  414.77    414.14        0.041548       -0.040243   
666     0.199959  414.77    414.14        0.035761       -0.045041   
...          ...     ...       ...             ...             ...   
1553    0.094023  414.01    414.01        0.076025       -0.019984   
3453    0.090075  414.01    414.01        0.127452        0.057028   
3574    0.089914  414.01    414.01        0.033826       -0.018993   
1600    0.091729  414.01    414.01        0.046547       -0.001848   
1416    0.093875  414.01    414.01        0.123175        0.048747   

      season_ch_9mon  season_ch_12mon  
1027       -0.041708         0.008545  
810        -0.046909         0.017417  
1485       -0.035907         0.001985  
124        -0.003790        -0.005347  
666        -0.018730        -0.001438  
...              ...              ...  
1553       -0.017610        -0.016625  
3453        0.034610         0.018882  
3574       -0.014658        -0.005164  
1600        0.003057         0.000032  
1416       -0.037019        -0.037368  

[298328 rows x 32 columns]

In [17]:
output = target_df.to_csv(index=False, encoding='utf-8', header=True)

blob_client = get_blob_client(filename=f'data_drift/target/{end_month}/{veg_type}.csv', conn_str=conn_str)
blob_client.upload_blob(output, overwrite=True, blob_type='BlockBlob')

{'etag': '"0x8D9EB57A7C0D902"',
 'last_modified': datetime.datetime(2022, 2, 8, 23, 6, 35, tzinfo=datetime.timezone.utc),
 'content_md5': None,
 'content_crc64': bytearray(b'\r\x98\xf5\x9a\x8c\x91\xa1\xab'),
 'client_request_id': 'c38ba678-8933-11ec-8b2c-1bbaf30259bc',
 'request_id': 'a105bcb5-e01e-000f-0240-1ded91000000',
 'version': '2020-06-12',
 'version_id': '2022-02-08T23:06:35.8771480Z',
 'date': datetime.datetime(2022, 2, 8, 23, 6, 35, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}